In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'

In [2]:
%%sql
SELECT name, type
FROM sqlite_master
WHERE type IN ("table","view");

Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


In [81]:
%%sql 
SELECT genre.name, COUNT(*) AS count, CAST(COUNT(*) AS float)/(SELECT CAST(COUNT(*) AS float) AS count
FROM genre
INNER JOIN track ON track.genre_id = genre.genre_id
INNER JOIN invoice_line ON track.track_id = invoice_line.track_id
INNER JOIN invoice ON invoice_line.invoice_id = invoice.invoice_id
WHERE invoice.billing_country = 'USA') AS count_percentage
FROM genre
INNER JOIN track ON track.genre_id = genre.genre_id
INNER JOIN invoice_line ON track.track_id = invoice_line.track_id
INNER JOIN invoice ON invoice_line.invoice_id = invoice.invoice_id
WHERE invoice.billing_country = 'USA'
GROUP BY genre.name
ORDER BY count_percentage DESC

Done.


name,count,count_percentage
Rock,561,0.5337773549000951
Alternative & Punk,130,0.12369172216936251
Metal,124,0.11798287345385347
R&B/Soul,53,0.05042816365366318
Blues,36,0.03425309229305423
Alternative,35,0.03330161750713606
Latin,22,0.02093244529019981
Pop,22,0.02093244529019981
Hip Hop/Rap,20,0.019029495718363463
Jazz,14,0.013320647002854425


In [98]:
%%sql
SELECT employee_id,hire_date,SUM(total) AS total
FROM employee
INNER JOIN customer ON customer.support_rep_id = employee.employee_id
INNER JOIN invoice ON invoice.customer_id = customer.customer_id
GROUP BY employee_id

Done.


employee_id,hire_date,total
3,2017-04-01 00:00:00,1731.5100000000039
4,2017-05-03 00:00:00,1584.0000000000034
5,2017-10-17 00:00:00,1393.920000000002


In [226]:
%%sql
CREATE VIEW count AS
SELECT country,COUNT(*) as count
FROM (SELECT customer.customer_id,SUM(total) AS total, country
FROM customer
INNER JOIN invoice ON customer.customer_id = invoice.customer_id
GROUP BY customer.customer_id)
GROUP BY country

Done.


[]

In [231]:
%%sql
CREATE VIEW total AS
SELECT country,SUM(total) AS total
FROM (SELECT customer.customer_id,SUM(total) AS total, country
FROM customer
INNER JOIN invoice ON customer.customer_id = invoice.customer_id
GROUP BY customer.customer_id)
GROUP BY country

Done.


[]

In [236]:
%%sql 
CREATE VIEW table1 AS
SELECT count.country, count, total, total/count AS average_value_of_sales
FROM count
INNER JOIN total ON count.country = total.country

Done.


[]

In [245]:
%%sql
CREATE VIEW count1 AS
SELECT country,COUNT(*) AS orders
FROM customer
INNER JOIN invoice ON customer.customer_id = invoice.customer_id
GROUP BY country

Done.


[]

In [252]:
%%sql
SELECT count1.country, count, total, average_value_of_sales, total/orders AS average_order_value
FROM table1 
INNER JOIN count1 ON count1.country = table1.country

Done.


country,count,total,average_value_of_sales,average_order_value
Argentina,1,39.6,39.6,7.92
Australia,1,81.18,81.18,8.118
Austria,1,69.3,69.3,7.699999999999999
Belgium,1,60.38999999999999,60.38999999999999,8.627142857142855
Brazil,5,427.67999999999995,85.53599999999999,7.011147540983606
Canada,8,535.59,66.94875,7.047236842105264
Chile,1,97.02000000000001,97.02000000000001,7.463076923076924
Czech Republic,2,273.24,136.62,9.108
Denmark,1,37.61999999999999,37.61999999999999,3.761999999999999
Finland,1,79.2,79.2,7.2
